# Targeting transition drivers in KIRC

In [ ]:
library('netZooR')
library('gplots')
library('reshape2')
library('ggplot2')

## 1. Identifying differential modules between normal and KIRC networks using ALPACA

In [ ]:
nTFs=1132

In [ ]:
df =read.table('alpaca_outputNet.csv',header=TRUE,sep=",")

In [ ]:
simp.alp <- alpaca(df,NULL,verbose=F)
simp.alp2 <- simp.alp[[1]]
simp.memb <- as.vector(simp.alp2)
names(simp.memb) <- names(simp.alp2)

simp.memb

In [ ]:
a=as.data.frame(simp.memb)

In [ ]:
a['ZNF180_A',]

In [ ]:
b=simp.alp[[2]]

In [ ]:
rownames(a)[1]

In [ ]:
for(i in 1:dim(a)[1]){
    print(i)
#for(i in 1:10){
    #print(rownames(a)[i])
    print(b[[rownames(a)[i]]])
}

In [ ]:
write.table(a,'contAlpaca.csv')

## 2. Estimating transition drivers using MONSTER

In [ ]:
kircnet=read.table('KRCC_contprior2.csv', header=T,sep=',',row.names = 1)
kirc_nat_net=read.table('NATKIRC_contprior2.csv', header=T,sep=',',row.names = 1)
healthynet=read.table('cortex_cont2.csv', header=T,sep=',',row.names = 1)

### 2.1 Healthy to KIRC transition

In [ ]:
interGenes = intersect(colnames(kircnet),colnames(healthynet))
indKircGene= match(interGenes, colnames(kircnet))
indHealthyGene= match(interGenes, colnames(healthynet))
set.seed(1619)
combinedRegNetworks_healthy_kirc=as.data.frame(cbind(healthynet[,indHealthyGene],kircnet[,indKircGene]))
nGenes=length(indHealthyGene)
design=c(rep(0,nGenes),rep(1,nGenes))

In [ ]:
monsterResRegNet <- monster(combinedRegNetworks_healthy_kirc, design ,motif=NA, nullPerms=1000, numMaxCores=12, mode='regNet')
saveRDS(monsterResRegNet, file="healthy_kirc_monster.rds")

### 2.2 NAT to KIRC transition

In [ ]:
interGenes = intersect(colnames(kircnet),colnames(kirc_nat_net))
indKircGene= match(interGenes, colnames(kircnet))
indHealthyGene= match(interGenes, colnames(kirc_nat_net))
set.seed(1619)
combinedRegNetworks_nat_kirc=as.data.frame(cbind(kirc_nat_net[,indHealthyGene],kircnet[,indKircGene]))
nGenes=length(indHealthyGene)
design=c(rep(0,nGenes),rep(1,nGenes))

In [ ]:
monsterResRegNet <- monster(combinedRegNetworks_nat_kirc, design ,motif=NA, nullPerms=1000, numMaxCores=12, mode='regNet')
saveRDS(monsterResRegNet, file="nat_kirc_monster.rds")

### 2.3 Healthy to NAT transition

In [ ]:
interGenes = intersect(colnames(kirc_nat_net),colnames(healthynet))
indKircGene= match(interGenes, colnames(kirc_nat_net))
indHealthyGene= match(interGenes, colnames(healthynet))
set.seed(1619)
combinedRegNetworks_healthy_nat=as.data.frame(cbind(healthynet[,indHealthyGene],kirc_nat_net[,indKircGene]))
nGenes=length(indHealthyGene)
design=c(rep(0,nGenes),rep(1,nGenes))

In [ ]:
monsterResRegNet <- monster(combinedRegNetworks_healthy_nat, design ,motif=NA, nullPerms=1000, numMaxCores=12, mode='regNet')
saveRDS(monsterResRegNet, file="healthy_nat_monster.rds")

### 2.4 Plots

In [ ]:
monsterResRegNet=readRDS("kircmonster.rds")

In [ ]:
monsterResRegNet

In [ ]:
monster.plot.monsterAnalysis(monsterResRegNet, rescale='magnitude', nTFs=70) # magnitude of null i sin the order of 2